In [17]:
import numpy as np
import sys 


from sympy import symbols, simplify, derive_by_array
from scipy.integrate import solve_ivp
from xLSINDy_sp import *
from sympy.physics.mechanics import *
from sympy import *
from Data_generator_py import image_process
import sympy
import torch;torch.random.manual_seed(66)
import sys
import HLsearch as HL
import example_pendulum
import time

In [18]:
save = False
environment = "server"
sample_size = 10
device = 'cuda:1'

In [19]:
sys.path.append(r'../../../HLsearch/')
#Saving Directory
momentum = True
params = {}
params['adding_noise'] = False
params['noise_type'] = 'image_noise'
params['noiselevel'] = 1e-1
params['changing_length'] = False
params['c'] = 1.4e-4
params['g'] = 9.81
params['l'] = 1
if environment == 'laptop':
    root_dir =R'C:\Users\87106\OneDrive\sindy\progress'
elif environment == 'desktop':
    root_dir = R'E:\OneDrive\sindy\progress'
elif environment == 'server':
    root_dir = R'/mnt/ssd1/stilrmy/Autoencoder-conservtive_expression'
x,dx,ddx,t = image_process(sample_size,params)
X = []
Xdot = []
for i in range(len(x)):
    temp_list = [float(x[i]),float(dx[i])]
    X.append(temp_list)
    temp_list = [float(dx[i]),float(ddx[i])]
    Xdot.append(temp_list)
X = np.vstack(X)
print(X.shape)
Xdot = np.vstack(Xdot)

type of t:  float64
(4998, 2)


In [20]:
temp = t
for i in range(sample_size-1):
    temp = np.append(temp,t)
print(temp.shape)
t = temp

(5000,)


In [21]:
states_dim = 2
states = ()
states_dot = ()
for i in range(states_dim):
    if(i<states_dim//2):
        states = states + (symbols('x{}'.format(i)),)
        states_dot = states_dot + (symbols('x{}_t'.format(i)),)
    else:
        states = states + (symbols('x{}_t'.format(i-states_dim//2)),)
        states_dot = states_dot + (symbols('x{}_tt'.format(i-states_dim//2)),)
print('states are:',states)
print('states derivatives are: ', states_dot)

states are: (x0, x0_t)
states derivatives are:  (x0_t, x0_tt)


In [22]:
#Turn from sympy to str
states_sym = states
states_dot_sym = states_dot
states = list(str(descr) for descr in states)
states_dot = list(str(descr) for descr in states_dot)

In [23]:
#build function expression for the library in str
expr= HL.buildFunctionExpressions(2,states_dim,states,use_sine=True)
#expr=['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
"a list of candidate function"
print(expr)
expr.pop(5)



['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']


'x0*x0_t'

In [24]:
Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot,device,scaling=False)
Eta = Eta.to(device)
Zeta = Zeta.to(device)
Delta = Delta.to(device)


In [25]:
#coefficients for the Lagrangian
mask = torch.ones(len(expr),device=device)
xi_L = torch.ones(len(expr), device=device).data.uniform_(-10,10)
prevxi_L = xi_L.clone().detach()
c = torch.ones(1,device=device)*-0.1
prec = c.clone().detach()

In [26]:
def loss(pred, targ):
    loss = torch.mean((pred - targ)**2) 
    return loss 

In [27]:
def clip(w, alpha):
    clipped = torch.minimum(w,alpha)
    clipped = torch.maximum(clipped,-alpha)
    return clipped

In [28]:
def proxL1norm(w_hat, alpha):
    if(torch.is_tensor(alpha)==False):
        alpha = torch.tensor(alpha)
    w = w_hat - clip(w_hat,alpha)
    return w

In [29]:
def training_loop(coef, c, prevcoef, prec, Zeta, Eta, Delta, xdot, t, bs, lr, lam):
    loss_list = []
    tl = xdot.shape[0]
    n = xdot.shape[1]
    momentum = True
    #if(torch.is_tensor(xdot)==False):
        #xdot = torch.from_numpy(xdot).to(device).float()
    v = coef.clone().detach().to(device).requires_grad_(True)
    c = c.clone().detach().to(device).requires_grad_(True)
    prev = prevcoef.clone().detach().to(device).requires_grad_(True)
    prec = prec.clone().detach().to(device).requires_grad_(True)
    q_tt_pred = 2
    for i in range(tl//bs):
        #computing acceleration with momentum
        if (momentum == True):
            vhat = (v + ((i - 1) / (i + 2)) * (v - prev)).clone().detach().requires_grad_(True)
            chat = (c + ((i - 1) / (i + 2)) * (c - prec)).clone().detach().requires_grad_(True)
        else:
            vhat = v.requires_grad_(True).clone().detach().requires_grad_(True)
            chat = c.requires_grad_(True).clone().detach().requires_grad_(True)
        prev = v
        prec = c
        #Computing loss
        zeta = Zeta[:,i*bs:(i+1)*bs]
        eta = Eta[:,i*bs:(i+1)*bs]
        delta = Delta[:,i*bs:(i+1)*bs]
        x_t = xdot[i*bs:(i+1)*bs,:]
        t_temp = t[i*bs:(i+1)*bs]
        t_temp = torch.from_numpy(t_temp).to(device).float()
        #forward
        temp = lagrangianforward(vhat,zeta,eta,delta,x_t,device)
        q_tt_pred = torch.exp(chat*t_temp)*temp
        q_tt_true = xdot[i*bs:(i+1)*bs,n//2:].T
        q_tt_true = torch.from_numpy(q_tt_true).to(device).float()
        lossval = loss(q_tt_pred, q_tt_true)
        lossval.requires_grad_(True)
        #Backpropagation
        lossval.backward()
        with torch.no_grad():
            v = vhat - lr * vhat.grad
            v = proxL1norm(v, lr * lam)
            vhat.grad = None
            c = chat - lr * chat.grad
            c = proxL1norm(c, lr * lam)
            c.grad = None
        loss_list.append(lossval.item())
    return v, c, prev, prec, torch.tensor(loss_list).mean().item(),tl

In [30]:
Epoch = 310
i = 1
lr = 1e-5
lam = 0.1
temp = 1000
while(i<=Epoch):
    xi_L , c, prevxi_L, prec, lossitem, q= training_loop(xi_L,c,prevxi_L,prec,Zeta,Eta,Delta,Xdot, t, 500,lr,lam)
    if i %20 == 0:
        print("\n")
        print("Stage 1")
        print("Epoch "+str(i) + "/" + str(Epoch))
        print("Learning rate : ", lr)
        print("Average loss : " , lossitem)
    if(temp <=5e-3):
        break
    if(temp <=1e-1):
        lr = 1e-5
    temp = lossitem
    i+=1



Stage 1
Epoch 20/310
Learning rate :  1e-05
Average loss :  48.40763854980469


Stage 1
Epoch 40/310
Learning rate :  1e-05
Average loss :  48.4074592590332


Stage 1
Epoch 60/310
Learning rate :  1e-05
Average loss :  48.40727615356445


Stage 1
Epoch 80/310
Learning rate :  1e-05
Average loss :  48.4070930480957


Stage 1
Epoch 100/310
Learning rate :  1e-05
Average loss :  48.40691375732422


Stage 1
Epoch 120/310
Learning rate :  1e-05
Average loss :  48.40673065185547


Stage 1
Epoch 140/310
Learning rate :  1e-05
Average loss :  48.406551361083984


Stage 1
Epoch 160/310
Learning rate :  1e-05
Average loss :  48.40636444091797


Stage 1
Epoch 180/310
Learning rate :  1e-05
Average loss :  48.40618133544922


Stage 1
Epoch 200/310
Learning rate :  1e-05
Average loss :  48.40599822998047


Stage 1
Epoch 220/310
Learning rate :  1e-05
Average loss :  48.40581512451172


Stage 1
Epoch 240/310
Learning rate :  1e-05
Average loss :  48.40563201904297


Stage 1
Epoch 260/310
Learning 

In [31]:
threshold = 1e-2
surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
expr = np.array(expr)[surv_index].tolist()

xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
prevxi_L = xi_L.clone().detach()
prec = c.clone().detach()

## obtaining analytical model
xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
c_cpu = np.around(c.detach().cpu().numpy(),decimals=4)
L = HL.generateExpression(xi_Lcpu,expr)
print(len(xi_L))
print('exp({}*t)'.format(c_cpu))

13
exp([-6.6494])


In [32]:
for stage in range(500):

    #Redefine computation after thresholding
    Zeta, Eta, Delta = LagrangianLibraryTensor(X,Xdot,expr,states,states_dot,device,scaling=False)
    Eta = Eta.to(device)
    Zeta = Zeta.to(device)
    Delta = Delta.to(device)

    #Training
    Epoch = 200
    i = 1
    lr = 1e-5
    if(stage==1):
        lam = 0
    else:
        lam = 0.1
    temp = 1000
    while(i<=Epoch):   
        xi_L , c, prevxi_L, prec, lossitem, q= training_loop(xi_L,c,prevxi_L,prec,Zeta,Eta,Delta,Xdot,t,500,lr,lam)
        if i %50 == 0:
            print("\n")
            print("Stage ",stage)
            print("Epoch "+str(i) + "/" + str(Epoch))
            print("Learning rate : ", lr)
            print("Average loss : " , lossitem)
        temp = lossitem
        if temp <= 0.5:
            lr = 2e-6
        if(temp <=1e-6):
            break
        i+=1
    
    ## Thresholding small indices ##
    threshold = 1e-1
    surv_index = ((torch.abs(xi_L) >= threshold)).nonzero(as_tuple=True)[0].detach().cpu().numpy()
    expr = np.array(expr)[surv_index].tolist()
    xi_L =xi_L[surv_index].clone().detach().requires_grad_(True)
    prevxi_L = xi_L.clone().detach()

    ## obtaining analytical model
    xi_Lcpu = np.around(xi_L.detach().cpu().numpy(),decimals=2)
    c_cpu = c.detach().cpu().numpy()
    L = HL.generateExpression(xi_Lcpu,expr)
    print("expression length:\t",len(xi_L))
    print("Result stage " + str(stage+2) + ":" , L)
    print('exp({}*t)'.format(c_cpu))



Stage  0
Epoch 50/200
Learning rate :  1e-05
Average loss :  47.57457733154297


Stage  0
Epoch 100/200
Learning rate :  1e-05
Average loss :  47.57412338256836


Stage  0
Epoch 150/200
Learning rate :  1e-05
Average loss :  47.573673248291016


Stage  0
Epoch 200/200
Learning rate :  1e-05
Average loss :  47.573219299316406
expression length:	 12
Result stage 2: 7.09*x0 + -7.29*x0_t + -4.89*sin(x0) + 8.98*cos(x0) + 1.34*x0**2 + 9.0*x0_t**2 + 8.68*x0*sin(x0) + 4.62*x0_t*sin(x0) + -7.06*sin(x0)**2 + 3.11*x0_t*cos(x0) + 0.69*sin(x0)*cos(x0) + 1.19*cos(x0)**2
exp([-6.6105])


Stage  1
Epoch 50/200
Learning rate :  1e-05
Average loss :  47.572975158691406


Stage  1
Epoch 100/200
Learning rate :  1e-05
Average loss :  47.57277297973633


Stage  1
Epoch 150/200
Learning rate :  1e-05
Average loss :  47.572566986083984


Stage  1
Epoch 200/200
Learning rate :  1e-05
Average loss :  47.57236099243164
expression length:	 12
Result stage 3: 7.1*x0 + -7.29*x0_t + -4.89*sin(x0) + 8.99*cos(x0) +